<a href="https://colab.research.google.com/github/changsksu/IMSE_Data_Science/blob/main/Hypothesis_Testings_z_t_F_paired_t_Chiefs_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This nb demonstrates One-Sample and Two-Sample hypothesis tests via the use of direct codes, statsmodels weightstats, and scipy ttest functions.




In addition, codes to compute critical values for z and t tests, confidence intervals, and pair-t-test are also included.

Ref. https://k-state.instructure.com/courses/144373/modules/items/4455306
Ref. https://www.kite.com/python/docs/scipy.stats.ttest_1samp


In [1]:
import math
import statistics
import numpy as np
import scipy.stats
import pandas as pd
from statsmodels.stats import weightstats as stests
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import t
import matplotlib.pyplot as plt

In [ ]:
# Use this cell and next if your data is on your local machine
# click on Choose File to read in a data file from you local machine
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving chiefs-vs-ravens.csv to chiefs-vs-ravens (1).csv
User uploaded file "chiefs-vs-ravens (1).csv" with length 121 bytes


In [ ]:
# pandas provides a better view of structured data with headings
# data is assigned to the pandas object called data1
import pandas as pd
data = pd.read_csv('chiefs-vs-ravens.csv', sep=',', na_values=".")
data.head()

,Date,Chiefs,Ravens
0,12/9/2018,27,24
1,9/22/2019,33,28
2,9/28/2020,34,20
3,9/19/2021,35,36
4,1/24/2024,17,10


In [3]:
# make sure that you use the raw format in Github
# the data is under Dr. Chang's github changsksu folder KState_IMSE541
# use the URL once you find or store the data file (use the raw tab)
# the following input statement is needed to use pandas dataframe feature
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/changsksu/KState_IMSE541/main/chiefs-vs-ravens.csv', sep=',', na_values=".")
# the foloowing statement only retrieves the first 5 obs
data

,Date,Chiefs,Ravens
0,12/9/2018,27,24
1,9/22/2019,33,28
2,9/28/2020,34,20
3,9/19/2021,35,36
4,1/24/2024,17,10
5,9/5/2024,27,20


# I. One-Sample Tests
**Ia Ztest when variance is known: Compute p-value via the scipy norm function**

In [4]:
Chiefs_mean=np.mean(data['Chiefs'])
Chiefs_mean

28.833333333333332

In [5]:
# np.std
# ddof is the degrees of freedom, which is 1 for sample standard deviation and 0 for population standard deviation
Chiefs_std=np.std(data['Chiefs'], ddof=1)
Chiefs_std

6.76510655249913

In [6]:
# assuming the variance of VIQ is known
# H0: mu = 28
# H1: mu > 28
# std=7 (known)
n=len(data['Chiefs'])
pval=1-norm.cdf((Chiefs_mean-28)/(7/np.sqrt(n)))
pval

0.38529397361274786

Since pval is larger than 0.05, we fail to reject H0. Another way to cite this result is that reject H0 at (1-pval)*100% significant level

**Ib. t test when variance is unknown: Compute p-value via the scipy t function**

In [14]:
# assuming the variance of the score is unknown
# H0: mu = 28
# H1: mu > 28
# std should be estimated
n=len(data['Chiefs'])
Chiefs_std=np.std(data['Chiefs'], ddof=1)
cdf = t.cdf((Chiefs_mean-28)/(Chiefs_std/np.sqrt(n)), df=(n-1))
pval=1-cdf
pval

0.3875019046961431

In [8]:
#Two-side p-value
2*pval

0.7750038093922862

# Analysis:
if one sided test H1>28, since pval is larger than 0.05, we cannot reject H0
if H1<28, then pval=norm.cdf((viqmean-28)/(viqstd/np.sqrt(n)))
if two sided test and pval is smaller than 0.025, we would have reject H0. Or multiply the pval*2 and use 0.05 as the criterion.
Replace norm.cdf by t.cdf if standard deviation is estimated

**Ic. using scipy ttest_1samp function**

In [9]:
# one sample t test (default two sided test)
# there are two ways to run this one sample t test
stats.ttest_1samp(data['Chiefs'], 28)
#scipy.stats.ttest_1samp(a=data['Chiefs'], popmean=28)

TtestResult(statistic=0.301730865061578, pvalue=0.7750038093922862, df=5)

Note that the default is two-side here. The method in 1b is one sided test

**Id. One Sample Test: using statsmodel's weightstats function**

ztest(x, y, value=28, alternative='two-sided')


*   x is the first sample
*   y is the second sample; set it at None for one sample test
*   alternative is the alternative hypothesis, which can be 'two-sided', 'larger', or 'smaller'; default alternative='two-sided'

Ref. https://www.statsmodels.org/dev/generated/statsmodels.stats.weightstats.ztest.html

In [11]:
# this function can also be used to test two samples!
# assuming the variance of the score is known or n > 30
# one-side H1 test is demonstrated here
ztest, propability_value = stests.ztest(data['Chiefs'], x2=None, value=28, alternative="larger")
print(float(propability_value))
if propability_value<0.05:
  print("Null hyphothesis rejected")
else:
  print("Cannot reject the Null hyphothesis")

0.38142861865579125
Cannot reject the Null hyphothesis


In [17]:
# this function can also be used to test two samples!
# please note that stests.ztest does not offer you a way enter given std
# assuming the variance of the score is known or n > 30
# two-sided test H1 is demonstrated here
ztest, propability_value = stests.ztest(data['Chiefs'], x2=None, value=28, alternative="two-sided")
print(float(propability_value))
if propability_value<0.05:
  print("Null hyphothesis rejected")
else:
  print("Cannot reject the Null hyphothesis")

0.7628572373115825
Cannot reject the Null hyphothesis


Please note that this result is slightly different from the "hand" calculation. The reason is that the "known" std is not provided when 1-sample z test is used.

In [15]:
# std.score is not computed but z test (not t-test) is used
# this result is close to the t test result when n is large (i.e.  > 30)
# in this case, std should be given since n<30
n=len(data['Chiefs'])
Chiefs_std=7
pval1 = norm.cdf((Chiefs_mean-28)/(Chiefs_std/np.sqrt(n)))
pval2=1-pval1
pval2

0.38529397361274786

**Until we understand more of the Statmodel weightstats function, we should use hand computations or scipy ttest!**

# II. Two Sample Tests

In [31]:
# Perform two-sided t-test
data1=data['Chiefs']
data2=data['Ravens']
t_stat, p_value_two_sided = stats.ttest_ind(data1, data2)
# the following is the z test results (You can use this when N >30)
ztest, propability_value = stests.ztest(x1=data1, x2=data2, value=0, alternative="larger", usevar="pooled")

# Convert to one-sided p-value for the alternative hypothesis:
# H0: mean(data1) == mean(data2)
# H1: mean(data1) > mean(data2)
p_value_one_sided= p_value_two_sided / 2

# If the alternative is mean(data1) < mean(data2), use:
# p_value_one_sided = 1 - (p_value_two_sided / 2)

In [27]:
#z test result
propability_value

0.09804799143321918

In [30]:
# t_test result
p_value_one_sided

0.11258263671345052

**Case I. Pooled variances of two populations**

**# Question: Is Chiefs better than Ravens in the Mahomes era? Six data points were given**

In [21]:
#using scipy
#two sample t-test 1: assume the variance is the same (default)
Chiefs=data['Chiefs']
Ravens=data['Ravens']
stats.ttest_ind(Chiefs, Ravens, equal_var=True)

TtestResult(statistic=1.292754496517956, pvalue=0.22516527342690104, df=10.0)

In [32]:
# H0: Chiefs = Ravens Ha: Chiefs > Ravens
# compute the p-value of one-sided test
t0, pval2=stats.ttest_ind(Chiefs, Ravens, equal_var=True)
pval1=pval2/2
# One sided test H1: Chiefs > Ravens
pval1

0.11258263671345052

from the above analysis, H0: Chiefs = Ravens vs H1: Chiefs > Ravens
this is a one-sided test. We would reject H0 at 89% significant.


# You turn: how do you answer the question: Is Ravens better than Chiefs in the Jackson era? (Hint: how to frame Ha?)

**Case II. Reject equal variance assumptions**

In [34]:
#two sample t-test method 2: the variances are not the same
stats.ttest_ind(Chiefs, Ravens, equal_var=False)
pval1=pval2/2
# One sided test H1: Chiefs > Ravens
pval1

0.11258263671345052

Analysis: in both cases, we reject H0 at 89% significant level! H0: Chiefs = Ravens

# III. F test
**To judge Case I or Case II should be used**

In [35]:
#F test for variances of two populations
# the following function is user defined
def f_test(x,y):
  x=np.array(x)
  y=np.array(y)
  f=np.var(x,ddof=1)/np.var(y, ddof=1) # cal F test statistic
  dfn=x.size-1
  dfd=y.size-1
  p=1-scipy.stats.f.cdf(f,dfn, dfd) #find p-value of F test statistics
  return f, p

f_test(Chiefs, Ravens)

(0.5990401396160558, 0.7062157442298039)

The first output is the F statistic and the second is the p-value. Since the p-value is large, we cannot reject H0: var1=var2. Therefore, the variances of two populations should be pooled.

# **IV. Paired T-Test**

In [36]:
#Paired t-Test method 1: using one sample t test
#Test if FISQ and PIQ are signficantly different
#The data for the pair (FISQ, PIQ) are obtained for the same subject
diff=data1 - data2
stats.ttest_1samp(diff, 0)


TtestResult(statistic=2.859645099213876, pvalue=0.0354211906599179, df=5)

The above p-value is large, which suggests H0 can not be rejected.

In [38]:
#Paired t-Test method 2: using repeated measures test
#The observations in data1 and data2 are paired

stats.ttest_rel(data1, data2)

TtestResult(statistic=2.859645099213876, pvalue=0.0354211906599179, df=5)

# Question: Is Chiefs better than Ravens in the Mahomes era? Is paired t test better than the two sample T-test?

In [40]:
# to answer this question, the spread is 0
spread=0
diff2=data['Chiefs']-data['Ravens']
stat3,pval3=stats.ttest_1samp(diff2, spread)
pval4=pval3/2
pval4

0.01771059532995895

When the spread is 0, then we will reject the hypothesis Chiefs and Ravens are equal at (1-0.0177)100% significant level. We are confident that Chiefs are better.

# **Question: Is Chiefs at least 3 points better than Ravens in the Mahomes era?**

In [41]:
# H0: mu_chiefs-mu_ravens=spread vs H1: mu_chiefs-mu_ravens <> spread
# note that the above H1 is a two sided test
spread=3
diff2=data['Chiefs']-data['Ravens']
# "`alternative` must be 'less', 'greater', or 'two-sided'."
stat5,pval5=stats.ttest_1samp(diff2, spread, alternative="two-sided")
pval6=pval5/2
print(f"p value: {pval6}")

#  another way to do the same computation
stats.ttest_1samp(diff2, spread, alternative="greater")

p value: 0.11176459569837864


TtestResult(statistic=1.3889704767610254, pvalue=0.11176459569837864, df=5)

As shown in the above computations, both methods generate the same  p-value.

# **How do you make a proper conclusion based on the p-values of spread=0 and 3?**

# **V. Critical Values of a t Distribution**

In [42]:
# critcal values of a t distribution
# for two-sided 95% CI, the left critical value can be optained
x=data['Chiefs']
df1=len(x) -1
stats.t.ppf(q=0.025, df=df1)


-2.5705818356363146

In [43]:
# critcal values of a t distribution
# for two sided 95% CI, the right critical value can be optained

stats.t.ppf(q=0.975, df=df1)


2.570581835636314

In [44]:
# critcal values of a t distribution
# for one sided 95% CI, the right critical value can be optained
# Ha u > u0

stats.t.ppf(q=0.95, df=df1)

2.0150483733330233

In [46]:
stats.t.cdf(2,6-1)

0.9490302605850709

# **VI. Construct a Confidence Interval of a Sample**

In [47]:
# 95% confidence interval (variance estimated)
x=data['Chiefs']
n=len(x)
df1=n-1
mean1=np.mean(x)
sigma1=np.std(x, ddof=1)/np.sqrt(n)
stats.t.interval(0.95, df=df1, loc=mean1, scale=sigma1)

(21.733789409446892, 35.93287725721977)

In [48]:
# hand calculation to verify the upper confidence interval limit

t0975=stats.t.ppf(q=0.975, df=df1)
UCL= mean1 + t0975 * sigma1
UCL

35.93287725721977

Your turn: can you compute the lower confidence interval limit?

In [49]:
# 95% confidence interval (variance known)
# known std=20
x=data['Chiefs']
mean1=np.mean(x)
stats.norm.interval(0.95, loc=mean1, scale=20/np.sqrt(n))

(12.830294412148966, 44.8363722545177)

Your turn: can you constuct a 99% CI of x?

# VII. Critical Value of Chi-square distribution

In [50]:
from scipy.stats import chi2

alpha = 0.05  # significance level
df = 10       # degrees of freedom

critical_value = chi2.ppf(1 - alpha, df)
print(f"Critical value: {critical_value}")



Critical value: 18.307038053275146


# VIII. Critical Value of F distribution

In [51]:
from scipy.stats import f

alpha = 0.05  # significance level
dfn = 5       # degrees of freedom for numeritor
dfd = 5       # degrees of freedom for denominator

# f.cdf(x, dfn, dfd) computes the CDF at value x with numerator df (dfn) and denominator df (dfd)

critical_value = f.ppf(1 - alpha, dfn, dfd)
print(f"Critical value: {critical_value}")

# assuming the ratio is F0, report the CDF value up to F0
F0=2
cdf_value = f.cdf(F0, dfn, dfd)
print(f"CDF value: {cdf_value}")

Critical value: 5.050329057632646
CDF value: 0.7674886808696213
